In [ ]:
## Tip: Please put code like this at the top of your notebook.
# We want all of the module/package installations to start up front

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py"
    import helper
    helper.easy_install()
else:
    sys.path.insert(0, '../')
    import helper
helper.set_plotting_style()

--2024-11-13 10:58:32--  https://raw.githubusercontent.com/ndcbe/optimization/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6493 (6.3K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   6.34K  --.-KB/s    in 0s      

2024-11-13 10:58:33 (69.4 MB/s) - ‘helper.py’ saved [6493/6493]

Installing idaes via pip...
idaes was successfully installed
idaes, version 2.6.0


Running idaes get-extensions to install Ipopt, k_aug, and more...
Checking solver versions:
Ipopt 3.13.2 (x86_64-pc-linux-gnu), ASL(20190605)



[K_AUG] 0.1.0, Part of the IDAES PSE framework
Please visit https://idaes.org/ (x86_64-pc-linux-gnu), ASL(20190605)



Couenne 0.5.8 -- an Open-Source solver for Mixed Integer Nonlinear Optimiz

In [ ]:
!pip install pyomo
from pyomo.environ import *
import matplotlib.pyplot as plt
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [ ]:
!apt install coinor-libipopt-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  coinor-libipopt1v5 libmumps-5.4 libmumps-dev libmumps-headers-dev libmumps-seq-5.4
  libmumps-seq-dev libscalapack-mpi-dev libscalapack-openmpi-dev libscalapack-openmpi2.1
  libscotch-6.1
Suggested packages:
  scalapack-doc
The following NEW packages will be installed:
  coinor-libipopt-dev coinor-libipopt1v5 libmumps-5.4 libmumps-dev libmumps-headers-dev
  libmumps-seq-5.4 libmumps-seq-dev libscalapack-mpi-dev libscalapack-openmpi-dev
  libscalapack-openmpi2.1 libscotch-6.1
0 upgraded, 11 newly installed, 0 to remove and 49 not upgraded.
Need to get 10.7 MB of archives.
After this operation, 64.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libscalapack-openmpi2.1 amd64 2.1.0-4 [1,534 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libmumps-5.4 amd64 5.4.1-2 [1,82

In [ ]:
!pip install cyipopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for cyipopt: filename=cyipopt-1.5.0-cp310-cp310-linux_x86_64.whl size=698718 sha256=416aad3c72b73c748eb020648799296bbac9cf63a7c957a76c08967ab676dc58
  Stored in directory: /root/.cache/pip/wheels/d2/18/c5/de38510f2a6606701875b6786d0961038bb557287270677535
Successfully built cyipopt


In [ ]:
!pip install ipopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for ipopt
  Running setup.py clean for ipopt
Failed to build ipopt
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (ipopt)


In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, minimize, NonNegativeReals, ConstraintList
import math

model = ConcreteModel()

T_inf = 65.6
k_air = 0.026
mu_air = 1.85e-5
Pr = 0.7189
rho_air = 1.177
u_inf = 0.8

model.x = Var(domain=NonNegativeReals, initialize=0.1)
model.T = Var(domain=NonNegativeReals, initialize=100)
model.delta_t = Var(domain=NonNegativeReals, initialize=0.016)

model.Re = Var(bounds=(1, 1000000), domain = NonNegativeReals)
model.Nu = Var()
model.h = Var()
model.Q = Var()

model.re_constraint = Constraint(expr=model.Re == (rho_air * u_inf * model.x) / mu_air)

model.nu_constraint = Constraint(expr=model.Nu == 0.332 * (((rho_air * u_inf * model.x) / mu_air)**0.5) * (Pr**(1/3)))

model.h_constraint = Constraint(expr=model.h == model.Nu * k_air / model.x)

model.delta_t_constraint = Constraint(expr=model.delta_t == 0.977 * 4.64 * model.x / (((rho_air * u_inf * model.x) / mu_air))**(1/2) * Pr**(1/3))

w = 1
model.q_constraint = Constraint(expr=model.Q == 2 * model.h * model.x * w * (model.T - T_inf))

model.obj_case1 = Objective(expr=model.delta_t, sense=minimize)

model.constraints = ConstraintList()
model.constraints.add(expr= (rho_air * u_inf * model.x) / mu_air >= 0)
model.constraints.add(expr= (rho_air * u_inf * model.x) / mu_air <= 50000)
model.constraints.add(expr=model.Q >= 140)
model.constraints.add(expr=model.Q <= 190)
model.constraints.add(expr=model.x >= 0.2)
model.constraints.add(expr=model.x <= 1)
model.constraints.add(expr=model.T >= 80)
model.constraints.add(expr=model.T <= 130)

solver = SolverFactory('cyipopt')

result_case1 = solver.solve(model, tee=True)
print("Case 1: Minimize delta_t")
print(f"Optimal x: {model.x()}")
print(f"Optimal T: {model.T()}")
print(f"Optimal Re: {(rho_air * u_inf * model.x()) / mu_air}")
print(f"Optimal Q: {model.Q()}")
print(f"Optimal delta_t: {model.delta_t()}")

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       13
Number of nonzeros in inequality constraint Jacobian.:        8
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        7
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        5
Total number of inequality constraints...............:        8
        inequality constraints with only lower bounds:        4
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        4

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, minimize, NonNegativeReals, ConstraintList
import math

model = ConcreteModel()

T_inf = 65.6
k_air = 0.026
mu_air = 1.85e-5
Pr = 0.7189
rho_air = 1.177
u_inf = 8

model.x = Var(domain=NonNegativeReals, initialize=0.1)
model.T = Var(domain=NonNegativeReals, initialize=100)
model.delta_t = Var(domain=NonNegativeReals, initialize=0.016)

model.Re = Var(bounds=(1, 1000000), domain = NonNegativeReals)
model.Nu = Var()
model.h = Var()
model.Q = Var()


model.re_constraint = Constraint(expr=model.Re == (rho_air * u_inf * model.x) / mu_air)

model.nu_constraint = Constraint(expr=model.Nu == 0.332 * (((rho_air * u_inf * model.x) / mu_air)**0.5) * (Pr**(1/3)))


model.h_constraint = Constraint(expr=model.h == model.Nu * k_air / model.x)

model.delta_t_constraint = Constraint(expr=model.delta_t == 0.977 * 4.64 * model.x / (((rho_air * u_inf * model.x) / mu_air))**(1/2) * Pr**(1/3))

w = 1
model.q_constraint = Constraint(expr=model.Q == 2 * model.h * model.x * w * (model.T - T_inf))

model.obj_case2 = Objective(expr=model.Q, sense=minimize)

model.constraints2 = ConstraintList()
model.constraints2.add(expr=(rho_air * u_inf * model.x) / mu_air >= 0)
model.constraints2.add(expr=(rho_air * u_inf * model.x) / mu_air <= 50000)
model.constraints2.add(expr=model.x >= 0.02)
model.constraints2.add(expr=model.x <= 1)
# model.constraints2.add(expr=model.delta_t >= 0.00001)
# model.constraints2.add(expr=model.delta_t <= 0.02)
model.constraints2.add(expr=model.T >= 80)
model.constraints2.add(expr=model.T <= 130)

solver = SolverFactory('cyipopt')

model.obj_case2.activate()
result_case2 = solver.solve(model, tee=True)  # Print solver output
print("\nCase 2: Minimize Q")
print(f"Optimal x: {model.x() * 10}")
print(f"Temperature (T): {model.T()}")
print(f"Optimal Re:  {model.Re()}")
print(f"Optimal Q: {model.Q()}")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.11.9, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       13
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        5

Total number of variables............................:        7
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equa